In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from time import time

# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [74]:
train = pd.read_csv('csv_files/modelo_final.csv')
test_final = pd.read_csv('csv_files/test_final.csv')

Transformamos las variables categóricas al tipo correspondiente, para reducir el uso de memoria

In [82]:
train['rango_edad'] = train['rango_edad'].astype('category')
train['sexo'] = train['sexo'].astype('category') 
train['nivel_estudios'] = train['nivel_estudios'].astype('category')
train['esta_estudiando'] = train['esta_estudiando'].astype('category')
train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
train['nivel_laboral'] = train['nivel_laboral'].astype('category')
train['nombre_zona'] = train['nombre_zona'].astype('category')
train['nombre_area'] = train['nombre_area'].astype('category')
train['sepostulo'] = train['sepostulo'].astype('category')

In [83]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')
test_final['nombre_area'] = test_final['nombre_area'].astype('category')

Nos guardamos los IDs necesarios para el submit

In [78]:
id_aviso_postulante = test_final['id']

Analizamos el set de entrenamiento y el test final

In [57]:
train.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo
1853774,1112461175,96LklB5,3.0,1,4.0,0.0,Analista de planeamiento comercial,10,3,2,Seguros,1.0
19104,1112393857,qe2L2b9,4.0,2,2.0,0.0,Clarkista con registro E2 Z/ San Miguel del Mo...,10,3,3,Producción,0.0
1884966,1111881762,mzd18aG,2.0,1,4.0,1.0,Asesor de Ventas en Posadas,9,2,3,Ventas,1.0
1552960,1112302480,2zP0wXx,1.0,2,4.0,1.0,COCINERO JUNIOR,10,3,3,Gastronomia,1.0
1094942,1112234457,mzdDG53,1.0,2,2.0,0.0,Administrativa contable,10,3,2,Administración,0.0


In [58]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 12 columns):
idaviso            int64
idpostulante       object
rango_edad         category
sexo               category
nivel_estudios     category
esta_estudiando    category
titulo             object
tipo_de_trabajo    category
nivel_laboral      category
nombre_zona        category
nombre_area        category
sepostulo          category
dtypes: category(9), int64(1), object(2)
memory usage: 97.3+ MB


In [59]:
test_final.sample(5)

,id,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area
67231,67231,1112460194,YO9Dxr,3.0,2.0,4.0,1.0,10,3,2,Calidad
58775,58775,1112453077,W9WPMwj,1.0,0.0,4.0,1.0,10,2,3,Producción
66974,66974,1112460141,NZvDwM,2.0,1.0,2.0,0.0,10,3,3,Administración
66805,66805,1112460108,JBxYp4e,2.0,2.0,3.0,1.0,10,3,3,Almacén / Depósito / Expedición
41172,41172,1112420559,1LXNoQ,4.0,2.0,4.0,0.0,10,3,3,Programación de producción


In [60]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
rango_edad         100000 non-null category
sexo               100000 non-null category
nivel_estudios     100000 non-null category
esta_estudiando    100000 non-null category
tipo_de_trabajo    100000 non-null category
nivel_laboral      100000 non-null category
nombre_zona        100000 non-null category
nombre_area        100000 non-null category
dtypes: category(8), int64(2), object(1)
memory usage: 3.2+ MB


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [87]:
# Features
f1 = 'rango_edad'
f2 = 'sexo'
f3 = 'nivel_estudios'
f4 = 'esta_estudiando'
f5 = 'tipo_de_trabajo'
f6 = 'nivel_laboral'
f7 = 'nombre_zona'
f8 = 'nombre_area'

features = [f1,f2,f3,f4,f5,f6]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [88]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['sepostulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [89]:
x_test_final = np.array(test_final[features])

# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [ ]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [5,10,20,30,40,50]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Valores grid search para K

In [ ]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Predecimos el test final con el mejor k obtenido

In [ ]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits/submit_knn.csv', index=False)

submit['sepostulo'].value_counts()

# 2. Random Forest

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [109]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit files

#### Submit con valores binarios

In [24]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_random_forest.csv', index=False)

In [2]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [26]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_random_forest_proba.csv', index=False)

In [3]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 3. Extra Trees

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [115]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [116]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [117]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_extra_trees.csv', index=False)

In [4]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_extra_trees_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [63]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [64]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [65]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_naive_bayes.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_naive_bayes_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 5. Perceptron

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

In [76]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_perceptron.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

# 6. Adaptive Boosting

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [30]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [31]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_ada_boost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [33]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_ada_boost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 7. Gradient Boosting

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':75, 'max_depth':4, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'subsample':1.0, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [122]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [123]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [124]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_gra_boost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_gra_boost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 8. Logistic Regression

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [37]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_log_reg.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [39]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_log_reg_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 9. Decision Tree

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [44]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_decision_tree.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [62]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 10. Bagging

## 10.1 Con Logistic Regression

### Set de entrenamiento

In [78]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [80]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [30]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_bagging_log_reg.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [91]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_bagging_log_reg_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

## 10.2 Con Decision Tree

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [73]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_bagging_decision_tree.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [75]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_bagging_decision_tree_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 11. Voting

### Set de entrenamiento

In [79]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeClassifier(**params_optimos_dt)
estimador_2 = GradientBoostingClassifier(**params_optimos_gb)
estimador_3 = ExtraTreesClassifier(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [81]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_voting.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

# 12. XGBoost

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con nuestro set de entrenamiento
xgboost.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred_final = xgboost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred_final)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = xgboost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = xgboost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_xgboost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_xgboost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")